In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_original= pd.read_csv("https://api.tournesol.app/exports/comparisons/")

In [ ]:
df=df_original.copy()

In [ ]:
id_user=pd.DataFrame(df["public_username"].unique())
id_user=id_user.reset_index()
id_user.columns=["user_id","public_username"]

In [ ]:
df=df.merge(id_user,how="left",on="public_username")

In [ ]:
id_entity_a=pd.DataFrame(df["video_a"].unique())
id_entity_b=pd.DataFrame(df["video_b"].unique())
id_entity= pd.concat([id_entity_a,id_entity_b],ignore_index=True)
id_entity_unique=pd.DataFrame(np.unique(id_entity.values))
id_entity_unique=id_entity_unique.reset_index()
id_entity_unique.columns=["entity_id","video"]

In [ ]:
id_comparison_unique=pd.DataFrame(np.vstack(list({tuple(row) for row in df[["public_username","video_a","video_b"]].values})))
id_comparison_unique=id_comparison_unique.reset_index()
id_comparison_unique.columns=["id_comparison","public_username","video_a","video_b"]

In [ ]:
#merge comparison
df=df.merge(id_comparison_unique,how="left",on=["public_username","video_a","video_b"])
#merge entity
df=df.merge(id_entity_unique,how="left",left_on="video_a",right_on="video")
df=df.drop(columns="video")
df=df.rename(columns={"entity_id":"entity_id_a"})
df=df.merge(id_entity_unique,how="left",left_on="video_b",right_on="video")
df=df.drop(columns="video")
df=df.rename(columns={"entity_id":"entity_id_b"})

In [ ]:
dfa=df[["user_id","entity_id_a"]]
dfa=dfa.rename(columns={"entity_id_a":"entity_id"})
dfb=df[["user_id","entity_id_b"]]
dfb=dfb.rename(columns={"entity_id_b":"entity_id"})
contributorrating_id=pd.DataFrame(np.unique(pd.concat([dfa,dfb],ignore_index=True).values,axis=0))
contributorrating_id=contributorrating_id.reset_index()
contributorrating_id.columns=['contributorrating_id','user_id','entity_id']


In [ ]:
df=df.merge(contributorrating_id,how="left",left_on=["user_id","entity_id_a"],right_on=["user_id","entity_id"])
df=df.drop(columns="entity_id")
df=df.rename(columns={"contributorrating_id":"contributorrating_id_a"})
df=df.merge(contributorrating_id,how="left",left_on=["user_id","entity_id_b"],right_on=["user_id","entity_id"])
df=df.drop(columns="entity_id")
df=df.rename(columns={"contributorrating_id":"contributorrating_id_b"})


In [ ]:
# 0 insert tournesol_poll
tournesol_poll=pd.DataFrame([(1,"videos","video","mehestan","true")], columns=['id', 'name', 'entity_type','algorithm','active'])
tournesol_poll
# 1 insert user_id with id_user
core_user=id_user.copy()
core_user["username"]=core_user.public_username.str.replace("'","")
core_user=core_user.drop(columns='public_username')
core_user.columns=['id', 'username']
core_user['password']="pbkdf2_sha256$260000$1Lucj6028OmJ3jO06B3TtR$C5MYoadWAuo2y16EvPL82v694FSl4tYYJBLHl+iW2zk="
core_user['is_superuser']="false"
core_user['email']=core_user.apply(lambda x:f"user{x.id}@tournesol.app",axis=1)
core_user['is_staff']="false"
core_user['is_active']="true"
core_user['is_demo']="false"
core_user['comment_anonymously']="false"
core_user['date_joined']="2020-05-28 06:06:28+00"
core_user['show_online_presence']="false"
core_user['show_my_profile']="true"
core_user['gender']="Not Specified"
core_user['nationality']="Not Specified"
core_user['residence']="Not Specified"
core_user['race']="Not Specified"
core_user['political_affiliation']="Not Specified"
core_user['religion']="Not Specified"
core_user['degree_of_political_engagement']="Not Specified"
core_user['moral_philosophy']="Not Specified"



# 2 insert entity with id_entity_unique

tournesol_entity=id_entity_unique.copy()
tournesol_entity["uid"]=tournesol_entity.apply(lambda x:  f"yt:{x.video}",axis=1)
tournesol_entity=tournesol_entity.drop(columns=['video'])
tournesol_entity.columns=['id','uid']
tournesol_entity['rating_n_ratings']=0 #TBD
tournesol_entity['rating_n_contributors']=0 #TBD
tournesol_entity['metadata']="{}"
tournesol_entity['type']='video'
tournesol_entity['metadata_timestamp']="2021-04-18 07:34:02.805316+00"
tournesol_entity['last_metadata_request_at']="2021-04-18 07:34:02.805316+00"


# 3 insert comparison with id_entity_unique df[user_id 	id_comparison 	entity_id_a 	entity_id_b]

tournesol_comparison=df[['user_id','id_comparison','entity_id_a','entity_id_b']]
tournesol_comparison=tournesol_comparison.drop_duplicates()
tournesol_comparison.columns=['user_id','id','entity_1_id','entity_2_id']
tournesol_comparison['poll_id']=1
tournesol_comparison['entity_1_2_ids_sorted']=tournesol_comparison.apply(lambda x: f"{x.entity_1_id}_{x.entity_2_id}",axis=1)
# 4 insert comparisoncriteriascore with df[criteria 	weight 	score  	id_comparison}

tournesol_comparisoncriteriascore=df[['criteria','weight','score','id_comparison']]
tournesol_comparisoncriteriascore=tournesol_comparisoncriteriascore.reset_index()
tournesol_comparisoncriteriascore.columns=['id','criteria','weight','score','comparison_id']

# 5 insert tournesol_contributorrating from contributorrating_id

tournesol_contributorrating=contributorrating_id.copy()
tournesol_contributorrating.columns=['id','user_id','entity_id']
tournesol_contributorrating['is_public']='true'
tournesol_contributorrating['poll_id']=1

In [ ]:
criteriarank_data_array=
[
    (1, 100, 'false', 1, 1),
    (2, 90, 'true', 2, 1),
    (3, 80, 'true', 3, 1),
    (4, 70, 'true', 4, 1),
    (5, 60, 'true', 5, 1),
    (6, 50, 'true', 6, 1),
    (7, 40, 'true', 7, 1),
    (8, 30, 'true', 8, 1),
    (9, 20, 'true', 9, 1),
    (10, 10, 'true', 10, 1)
]
tournesol_criteriarank=pd.DataFrame(criteriarank_data_array, columns=['id','rank','optional','criteria_id','poll_id'])

In [ ]:
core_user[core_user.username.str.contains("Falc")].username

In [ ]:
#https://stackoverflow.com/questions/31071952/generate-sql-statements-from-a-pandas-dataframe
def SQL_INSERT_STATEMENT_FROM_DATAFRAME(SOURCE, TARGET):
    sql_texts = []
    for index, row in SOURCE.iterrows():       
        sql_texts.append('INSERT INTO '+TARGET+' ('+ str(', '.join(SOURCE.columns))+ ') VALUES '+ str(tuple(row.values)))        
    return sql_texts

In [ ]:
sql_commands=list()
sql_commands.extend(SQL_INSERT_STATEMENT_FROM_DATAFRAME(tournesol_poll,"public.tournesol_poll"))
sql_commands.extend(SQL_INSERT_STATEMENT_FROM_DATAFRAME(core_user,"public.core_user"))
sql_commands.extend(SQL_INSERT_STATEMENT_FROM_DATAFRAME(tournesol_entity,"public.tournesol_entity"))
sql_commands.extend(SQL_INSERT_STATEMENT_FROM_DATAFRAME(tournesol_comparison,"public.tournesol_comparison"))
sql_commands.extend(SQL_INSERT_STATEMENT_FROM_DATAFRAME(tournesol_comparisoncriteriascore,"public.tournesol_comparisoncriteriascore"))
sql_commands.extend(SQL_INSERT_STATEMENT_FROM_DATAFRAME(tournesol_contributorrating,"public.tournesol_contributorrating"))
sql_commands.extend(SQL_INSERT_STATEMENT_FROM_DATAFRAME(tournesol_criteriarank,"public.tournesol_criteriarank"))



In [ ]:
df2=pd.DataFrame(sql_commands)

In [ ]:
df2.to_csv("my_csv.sql",sep=";")

In [ ]:
# https://www.geeksforgeeks.org/how-to-insert-a-pandas-dataframe-to-an-existing-postgresql-table/

import psycopg2
import psycopg2.extras as extras


def execute_values(conn, df, table):

	tuples = [tuple(x) for x in df.to_numpy()]

	cols = ','.join(list(df.columns))
	# SQL query to execute
	query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
	cursor = conn.cursor()
	try:
		extras.execute_values(cursor, query, tuples)
		conn.commit()
	except (Exception, psycopg2.DatabaseError) as error:
		print("Error: %s" % error)
		conn.rollback()
		cursor.close()
		return 1
	print("the dataframe is inserted")
	cursor.close()


def truncate_cascade(conn, tablename):
    curs = conn.cursor()
	try:
		curs.execute("TRUNCATE TABLE %s CASCADE" % tablename)
        curs.close()
        conn.commit()
	except (Exception, psycopg2.DatabaseError) as error:
		print("Error: %s" % error)
		conn.rollback()
		cursor.close()
		return 1


In [ ]:
conn = psycopg2.connect(
	database="tournesol", user='tournesol', password='password', host='127.0.0.1', port='5432'
)




In [ ]:
truncate_cascade(conn,"public.core_user")
truncate_cascade(conn,"public.tournesol_poll")
truncate_cascade(conn,"public.tournesol_entity")

In [ ]:
execute_values(conn, tournesol_poll, 'public.tournesol_poll')
execute_values(conn,core_user,"public.core_user")
execute_values(conn,tournesol_entity,"public.tournesol_entity")
execute_values(conn,tournesol_comparison,"public.tournesol_comparison")
execute_values(conn,tournesol_comparisoncriteriascore,"public.tournesol_comparisoncriteriascore")
execute_values(conn,tournesol_contributorrating,"public.tournesol_contributorrating")
execute_values(conn,tournesol_criteriarank,"public.tournesol_criteriarank")